In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from scipy.special import binom
from tqdm import tqdm_notebook as tqdm
%matplotlib inline

## NAEC Workshop
### José Moran, Michael Benzaquen

The model is defined as follows:
* There are N agents in total, each of which makes a choice $S_i=\pm1$ with the positive choice corresponding to "the right choice".
* A fraction $z$, or an amount $N_i= \lfloor z\cdot N\rfloor$ among them are informed, and make their mind based on a model that works better than randomness, so they pick $S_i = +1$ with a probability $p>1/2$.
* The followers, making up a fraction $1-z$ corresponding to $N_f$ individuals, make their mind at first purely randomly, and then follow this algorithm:

    1. Agent $i$ polls a group $g$ of $m$ individuals, informed or not.
    2. Agent $i$ picks the choice of the majority, setting $$ S_i = \mbox{Sign}\left(\sum_{k\in g}S_k\right)$$
    
    
We model this with an array $S_i$ where $0\leq i < N_i$ corresponds to the informed agents, while $N_i\leq i<N$ corresponds to followers.


We then focus on $q_t$, the probability that a **follower** has made the right choice at time $t$, and $\pi_t$, the probability that **any individual** has made the right choice at time $t$.

Therefore,
$$q_t = \frac{1}{N_f}\sum_{i\in \text{fol.}} (S_i>0)$$
$$\pi_t = \frac{1}{N}\sum_{0\leq i <N} (S_i>0)$$

In [ ]:
class MarsiliCurty(object):
    
    def __init__(self, N, z, p, m):
        #set the parameters
        self.N_i = int(N * z)
        self.N_f = N - self.N_i
        self.N = N
        self.p = p
        self.m = m
        #define the array of choices
        self.S = np.empty(N)
        #pick the initial choices for informed and non-informed agents
        self.S[:self.N_i] = np.random.choice([-1,1],
                                                      p = [1-p,p],
                                                      size = self.N_i)
        self.S[self.N_i:] = np.random.choice([-1,1],
                                                   p = [0.5, 0.5],
                                                  size = self.N_f)
    

    def compute_q(self):
        followers = self.S[self.N_i:]
        return np.mean(followers > 0)
    
    def compute_pi(self):
        return np.mean(self.S > 0)
    
    def compute_informed(self):
        informed = self.S[:self.N_i]
        return np.mean(informed > 0)
            
    def time_step(self):
        #pick a random follower, i.e. pick 
        #i in [N_i,N_i+1,...,N]
        random_follower = np.random.randint(low = self.N_i, high = self.N)
        #pick a group of m individuals you want to poll
        group = np.random.choice(self.N, size = self.m)
        #get the choices of the group
        group_choices = self.S[group]
        #align your choice with that of the majority
        avg_group_choice = np.mean(group_choices)
        self.S[random_follower] = np.sign(avg_group_choice)
        
        
    def iterate(self,T):
        #run a simulation during T timesteps
        #returns the evolution of q and pi 
        q = np.empty(T)
        pi = np.empty(T)
        for t in range(T):
            q[t] = self.compute_q()
            pi[t] = self.compute_pi()
            self.time_step()
        return q, pi 

We first check our code, and see how its initial values are set:

In [ ]:
N = 300
z = 0.3
p = 0.52
m = 12

MC = MarsiliCurty(N, z, p, m)
print("Checking initial values:")
print(f"Chosen value of p={p}")
print(f"Fraction of people who are right: pi={MC.compute_pi()}")
print(f"Fraction of followers who are right: q={MC.compute_q()}")
print(f"Fraction of informed agents who are right: q={MC.compute_informed()}")

We now run a simulation with these values, during $T=4000$ timesteps.

In [ ]:
%time q, pi = MC.iterate(4000)

In [ ]:
plt.figure(figsize = (8,8))
plt.plot(q, label='$q$')
plt.plot(pi, label=r'$\pi$')
plt.legend(fontsize = 15)
plt.xlabel('t')

We therefore define the average value of $q$ as the value observed during the $1000$ last time-steps of the simulation. 
We will plot this against $z$ for fixed values of $z$,$p$ and $m$.

In [ ]:
def get_q(z, p=0.52, m=11):
    #create an instance of the simulation
    MC = MarsiliCurty(N, z, p, m)
    #run the simulation and return the average of the 1000 last values
    q, pi = MC.iterate(4000)
    return np.mean(q[:-100])

In [ ]:
zs = np.linspace(0,1,50, endpoint=False)
qs = np.array([get_q(z) for z in tqdm(zs)])

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(zs,qs, label='Simulation results')
plt.xlabel(r'$z$', fontsize = 22)
plt.ylabel(r'$q$', fontsize = 22)


It's actually possible to understand this analytically...

In [ ]:
import scipy.optimize as opt

In [ ]:
def fixed_point(z,p,m, n_its = 200):
    def F_z(q):
        pi = z * p + (1-z)*q
        F_z = np.sum([binom(m, l)* pi**l * (1-pi) ** (m-l) for l in range((m+1)//2, m+1)])
        return F_z
    
    q_0 = 0 
    q_1 = 1
    q_mid = 0.5
    for i in range(n_its):
        q_0 = F_z(q_0)
        q_1 = F_z(q_1)
    q_mid = opt.brentq(lambda q: F_z(q)-q, q_0+0.1, q_1-0.1)
    return q_0, q_1, q_mid

In [ ]:
lower = np.empty(len(zs))
upper = np.empty(len(zs))
mid = np.empty(len(zs))

for i in tqdm(range(len(zs))):
    lower[i], upper[i], mid[i] = fixed_point(zs[i], p =0.5001, m =10)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(zs, lower, color='red', label='Analytical lower branch')
plt.plot(zs, upper, label='Analytical upper branch')
plt.plot(zs, mid, '--', label='Analytical mid branch', color='black')

plt.scatter(zs,qs, label='Simulation results')
plt.xlabel(r'$z$', fontsize = 22)
plt.ylabel(r'$q$', fontsize = 22)
plt.legend()